# 5章　誤差逆伝播法

4章では勾配を数値微分によって求めていたが、この章では、重みパラメータの勾配の計算を効率よく行う手法である**「誤差逆伝播法」**について学んでいく。

数式を使って理解していく方法と、計算グラフを使って理解していく2つの方法をやっていこう。

## 5.1 計算グラフ<123>


### 5.1.1 計算グラフを使って解く<124>

計算グラフを使って問題を解くには以下の手順で進む

1:計算グラフを構築する

2:計算グラフ上で計算を左から右へ進める。(この左から右へ進めるというのは**順伝播**という。)

実際に以下の簡単な2つの問に対して計算グラフを解いてみよう

問1:太郎君はスーパーで1個100円のリンゴを2個買いました。支払う金額は?消費税は10%とする。

問2:太郎君はスーパーで1個100円のリンゴを2個,1個150円のミカンを3個買いました。支払う金額は?消費税は10%とする。

### 5.1.2 局所的な計算<126>

局所的な計算とは、全体でどのようなことが行われていても、「自分に関係する小さな範囲」だけから次の結果を出力できるということ。

実際に例を出してみる。

図5-4

この図においてリンゴとそれ以外の買い物を合計する計算(4000 + 200)は、4000という数字がどのように計算されたかに関して考えずに、ただ2つの数を足している。
これが、局所的なということ。

### 5.1.3 なぜ計算グラフで解くのか<127>

計算グラフを使って解くメリットは以下

・局所的な計算

各ノードでは単純な計算に集中して問題を単純化できる

・途中計算をすべて保持できる。

・逆方向の伝播によって微分を効率的に行える←**最大のメリット**

上で使った問１を用いて、3つ目のメリット逆伝播の微分を効率的に行えるについて考察していこう。




問１に対してりんごの値段が値上がりした場合、最終的な支払い金額にどのように影響するのかを調べる。これはすなわち「りんごの値段に対する支払い金額の微分」とみることができる。

図５ー５

逆伝播は右から左へ「1→1.1→2.2」となっている。（支払い金額の割合とれば求めれる）

すなわち、りんごの値段が微小変化したら、その変化の2.2倍だけ増加します。

## 5.2 連鎖律<129>

### 5.2.1 計算グラフの逆伝播<129>

計算グラフを用いた逆伝播の例は以下である。


逆伝播の計算手順は、信号$E$に対して、ノードの局所的な微分をかけて、それを次のノードに伝播していく。

### 5.2.2 連鎖律とは<129>

### 5.2.3 連鎖律と計算グラフ<131>

次の式(5.1)に関する連鎖律の計算を計算グラフで表すことを考える。

$$z = t^2\\t = x + y \\ (5.1)$$

chain ruleを用いて、$\frac{\partial z}{\partial x}$を求めた結果が以下である。

$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t*1 = 2(x + y)$

これを図式化したのが以下である。

図5-6

結局何が言いたいかって、一番左の逆伝播の結果。

これは連鎖律より$\frac{\partial z}{\partial z}\frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = \frac{\partial z}{\partial x} $がなりたち、「xに関するzの微分」に対応している。

ここまでの計算結果を用いれば計算グラフは以下となる

図5-8

## 5.3 逆伝播<132>

「＋」や「×」といった演算に対して逆伝播が成り立つ仕組みに関して考察していこう。

### 5.3.1 加算ノードの逆伝播<132>

$z = x + y$という式に対して逆伝播を見ていこう。

まずzに関してx,yそれぞれの偏微分を考えると

$$\frac{\partial z}{\partial x} = 1,\frac{\partial z}{\partial y} = 1 \hspace{10mm}(5.5)$$
となる

ここで、最終的に$L$という値を出力する大きな計算グラフを考えると、以下のようになる。

図5-9

このように加算ノードの逆伝播は１を乗算するだけなので、入力値をそのまま次のノードに流すだけ。

### 5.3.2 乗算ノードの逆伝播<134>

次に乗算ノードの逆伝播として

$z = xy$という式を考える。この式の微分は以下である。

$$\frac{\partial z}{\partial x} = y, \frac{\partial t}{\partial y} = x \hspace{10mm} (5.6)$$

この乗算の逆伝播を図に表すと以下になる。

図5-12

具体的に「5×10=50」という計算に対して乗算ノードは以下となる。

図5-13

つまりひっくり返した形だな。加算の時はそのまま上流の値を流すだけだったので、入力信号の値とか要らなかったけど、乗算の場合はひっくり返すから入力値必須。

### 5.3.3 りんごの例<135>

改めて最初の２個のリンゴと消費税の問いに対して、この加算ノードと乗算ノードを用いて計算グラフを現そう。

図5-14

## 5.4単純なレイヤの実装<137>

計算グラフの乗算ノードを「乗算レイヤ(MulLayer)」、加算ノードを「加算レイヤ(AddLayer)」として実装していこう。

### 5.4.1 乗算レイヤの実装<137>

In [94]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  #順伝播に対応
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y

    return out

  #逆伝播に対応
  def backward(self, dout): #doutは上流から伝わってきた微分
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

では、この上さんレイヤを用いて、以下の図の支払額を順伝播、各変数に関する微分を逆伝播で求める。

図５−１６

In [95]:
#初期化
apple = 100
apple_num = 2
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#支払額を順伝播
apple_price = mul_apple_layer.forward(apple, apple_num) #順伝播の１個目のノード
price = mul_tax_layer.forward(apple_price, tax) #順伝播の2個目のノード

print(f"支払額: {price}")

#各変数に関する微分を逆伝播
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)

print(f"りんご微分: {dapple},りんごの個数微分: {dapple_num},消費税微分: {dtax}")

支払額: 220.00000000000003
りんご微分: 2.2,りんごの個数微分: 110.00000000000001,消費税微分: 200


### 5.4.2 加算レイヤの実装<139>

In [96]:
class AddLayer:
  def __init__(self):
    pass

  #順伝播に対応
  def forward(self, x, y):
    out = x + y
    return out

  #逆伝播に対応
  def backward(self, dout): #doutは上流から伝わってきた微分
    dx = dout * 1
    dy = dout * 1

    return dx, dy

加算レイヤは入力値によらないのだったから、initはpassでいいよね

加算レイヤは以下のやつで実装していこう

図5-17

In [97]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#支払額を順伝播
apple_price = mul_apple_layer.forward(apple, apple_num) 
orange_price = mul_orange_layer.forward(orange, orange_num) 
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#各変数に関する微分を逆伝播
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(f"支払額: {all_price}")
print(f"""りんご微分: {dapple},りんごの個数微分: {dapple_num},
          ,みかん微分: {dorange},みかんの個数微分: {dorange_num}
          ,消費税微分: {dtax}""")

支払額: 650
りんご微分: 2.2,りんごの個数微分: 110.00000000000001,
          ,みかん微分: 3.3000000000000003,みかんの個数微分: 165.0
          ,消費税微分: 650


## 5.5活性化レイヤの実装<141>

計算グラフの考え方をNNに適応していきたいと思う。

### 5.5.1 ReLUレイヤ<141>

ReLUは次の式で表されるんだった。
$$ y =  \left\{ \begin{matrix}x&(x > 0)\\ 0&(x \leq 0) \end{matrix} \right. \hspace{10mm}(5.7)$$

式(5.7)から、xに関するyの微分は次のようにかける

$$\frac{\partial y}{\partial x} = \left\{ \begin{matrix}1&(x > 0)\\ 0&(x \leq 0) \end{matrix} \right. \hspace{10mm}(5.8)$$


ReLUレイヤの計算グラフ

図５−１８

In [98]:
class ReLU:
  def __init__(self):
    self.mask = None #maskはtrue/FalseからなるNumpy配列
  
  def forward(self, x):
    self.mask = (x <= 0) 
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    
    return dx

### 5.5.2 Sigmoidレイヤ<143>

シグモイド関数は以下式(5.9)
$$y = \frac{1}{1 + exp(-x)} \hspace{10mm}(5.9)$$

この式を計算グラフで表すと以下である

図5-19

このシグモイドレイヤでは今までの「乗算レイヤ」「加算レイヤ」に加えて、「exp」「/」が入ってきているので、この二つを先に処理しよう。



・「/」ノード

割り算は$y = \frac{1}{x}$でかけることから、

$\frac{\partial y}{\partial x} = -\frac{1}{x^2} = -y^{2}$

従って逆伝播は、上流の値に対して$-y^{2}$を乗算して下流に流せばOK

・「exp」ノード



expノードでは$y = exp(x)$を表し、その微分は以下でかける。

$$\frac{\partial y}{\partial x} =exp(x) \hspace{10mm}(5.11)$$

仕方って、逆伝播は$exp(x)$を乗算して下流に流せばOK

これらのことを用いるとシグモイドレイヤの計算グラフは以下となる。

図5-20

簡略化すると以下となる

図5-21

さらにシグモイドレイヤの逆伝播を微分すると、(yにシグモイド関数を代入したりする。)

$\frac{\partial L}{\partial y}y^{2}exp(-x) = \frac{\partial L}{\partial y}y(1-y) \hspace{10mm}(5.12)$

この計算より、シグモイドレイヤの逆伝播は順伝播の出力にしか依存しない。

図5-21

In [99]:
class Sigmoid:
  def __init__(self):
    self.out = None #順伝播での出力を保存する。

  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out
  
  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out

    return dx

## Affine/Softmaxレイヤの実装<147>

### 5.6.1Affineレイヤ<147>

NNの順伝播で行う行列の積は、幾何学では**「アフィン変換」**と呼ばれる。

そのアフィン変換を行う処理を「Affineレイヤ」という名前で実装していこう。


これまでスカラ値で行ってきた計算グラフを「行列」がノード間を伝播する場合についても考えていこう。

行列を対象とした逆伝播でも、行列の要素毎に書き出すことで、今までのスカラー値を対象とした計算グラフと同じ手順で求めることができる。

図5-25

### 5.6.2バッチ版Affineレイヤ<150>



ここまでは、入力が$X$の一つのデータを対象にしていたが、こっからは$N$個のデータをまとめて順伝播することを考える。

図５−２７

In [100]:
class Affine():
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x = x
    out = np.dot(x, self.W) + self

    return out

  def backward(self, dout): #doutは上流からの微分
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis=0)
    
    return dx


### 5.6.3 Softmax-with-Lossレイヤ<152>

出力層であるsoftmaxをAffine,ReLUと共に表すと以下図のようになる。

ず５−２８

NNの正規化しない出力結果は「スコア」と呼ばれます。

Softmaxレイヤを実装していくが、損失関数(交差エントロピー誤差)も含めて「Softmax-with-Loss」という名前で実装していく。

計算グラフは以下である。

ず５−２９

簡易版は以下

ず５−３０

これは、３クラス分類を考えてる。

注目すべきは、softmaxからの逆伝播が「y - t」とめちゃ綺麗。(逆に言えばこうなるように設定されているんだけどね。)

具体例を考えよう。

教師ラベルが(0, 1, 0) であるデータに対して

Softmaxレイヤの出力が(0.3, 0.2. 0.5)であったとする。

この場合、softmaxレイヤからの逆伝播は(0.3, -0.8, 0.5)という大きな誤差を伝播する。故にsoftmaxレイヤよりも前のレイヤは大きな誤差から大きな内容を学習する。

一方、

教師ラベルが(0, 1, 0) であるデータに対して

Softmaxレイヤの出力が(0.01, 0.99, 0.0)であったとする。

この場合、softmaxレイヤからの逆伝播は(0.01, -0.01, 0)という小さな誤差を伝播する。故にsoftmaxレイヤよりも前のレイヤは小さな誤差から小さな内容を学習する。

In [101]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None
    self.y = None #softmaxの出力
    self.t = None #教師データ

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)

    return self.loss

  def backward(self, dout=1):
    batch_size = self.shape[0]
    dx = (self.y - self.t) / batch_size

    return dx

## 誤差逆伝播法の実装<156>


### 5.7.1NNの学習の全体図<156>

NNの学習手順


・step1(ミニバッチ)

訓練データの中からランダムに一部のデータを選びだす。(標本抽出的な)

・step2(勾配の計算)

ミニバッチの損失関数を減らすため、各重みパラメータを求める。

・step3(パラメータの更新)

重みパラメータを勾配方向に更新していく

以上、step1~step3を繰り返す。

### 5.7.2誤差逆伝播法に対応したNNの実装<157>

4.5で作ったTwoLayerNetに加えていく形で実装していく

In [102]:
import sys,os
sys.path.append("/content/drive/MyDrive/DS/deep_learning/deep-learning-from-scratch-master")
from common.functions import *
from common.gradient import numerical_gradient
from collections import OrderedDict #順番つきディクショナリ

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #重みの初期化
    self.params = {}
    self.params['W1'] = weight_init_std * \
                       np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * \
                       np.random.randn(input_size, hidden_size)
    self.params['b2'] = np.zeros(hidden_size)  

    #レイヤの生成
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['ReLU1'] = ReLU()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()

  #認識(推論)を行う。xは画像データ
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)

      return x

  #x:入力データ, t:教師データ
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)

  #認識精度を求める
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  #重みパラメータに対する勾配を計算する。
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads["W1"] = numerical_gradient(loss_W, self.params['W1'])
    grads["b1"] = numerical_gradient(loss_W, self.params['b1'])
    grads["W2"] = numerical_gradient(loss_W, self.params['W2'])
    grads["b2"] = numerical_gradient(loss_W, self.params['b2'])

    return grads

  def gradient(self, x, y):
    #forward
    self.loss(x, t)

    #backward
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    grads = {}
    grads["W1"] = self.layers['Affine1'].dW
    grads["b1"] = self.layers['Affine1'].db
    grads["W2"] = self.layers['Affine2'].dW
    grads["b2"] = self.layers['Affine2'].db

    return grads

ModuleNotFoundError: ignored

レイヤを生成したことは効果絶大！なぜなら、大きなネットワークと作りたければ、このレイヤーを追加するだけでいいのだから。

### 5.7.3誤差逆伝播法の勾配確認<160>

誤差逆伝播法は、効率がいいがミスが発生しがち

対して、数値微分は、実装は簡単だが、効率が悪い。

本当は誤差逆伝播だけでいきたいところだけど、そんな感じでミスが多いから数値微分を用いて**勾配確認**をする

In [ ]:
import sys,os
sys.path.append("/content/drive/MyDrive/DS/deep_learning/deep-learning-from-scratch-master")
from dataset.mnist import load_mnist
from ch04.two_layer_net import TwoLayerNet

(X_train, t_train), (X_test, t_test) = \
  load_mnist(flatten=True, normalize=False)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

#勾配の計算
grad_numerical = network.numerical_gradient(x_batch, t_batch) #数値計算勾配
grad_backprop = network.gradient(x_batch, t_batch) #誤差逆伝播勾配
  
#各重みの絶対誤差の平均
for key in grad_numerical.keys():
  diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
  print(key + f": {str(diff)}")

### 5.7.4誤差逆伝播法を使った学習<162>

In [ ]:
#学習を行う過程で定期的に訓練データとテストデータを対象に認識精度を記録
import sys,os
sys.path.append("/content/drive/MyDrive/DS/deep_learning/deep-learning-from-scratch-master")
from dataset.mnist import load_mnist
from ch04.two_layer_net import TwoLayerNet

(X_train, t_train), (X_test, t_test) = \
  load_mnist(flatten=True, normalize=False)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

#ハイパーパラメータ
iters_num = 10000 #繰り返し回数(iteration)
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []
iter_per_epoch = max(train_size / batch_size, 1) #1epochあたりの繰り返し回数


for i in range(iters_num):
  #ミニバッチの取得
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  #勾配の計算(誤差逆伝播)
  grad = network.gradient(x_batch, t_batch) # 高速版
  
  #パラメータの更新
  for key in ("W1", "b1","W2", "b2"):
    network.params[key] -=learning_rate * grad[key]

  #学習経過の記録
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)
  
  #1epochごとの認識精度を計算
  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test.acc)
    print("train acc, test acc |" + str(train_acc) + "," + str(test_acc))